### Import Libraries

In [1]:
import cv2
import numpy as np
from glob import glob
import os
from PIL import Image
import shutil
import matplotlib.pyplot as plt

### convert labels helper function

In [ ]:
def convert_labels(size, x1, y1, x2, y2):
    """
    Definition: Parses label files to extract label and bounding box
        coordinates.  Converts (x1, y1, x1, y2) KITTI format to
        (x, y, width, height) normalized YOLO format.
    """
    def sorting(l1, l2):
        if l1 > l2:
            lmax, lmin = l1, l2
            return lmax, lmin
        else:
            lmax, lmin = l2, l1
            return lmax, lmin
    
    xmax, xmin = sorting(x1, x2)
    ymax, ymin = sorting(y1, y2)
    dw = 1./size[0]
    dh = 1./size[1]
    x = (xmin + xmax)/2.0
    y = (ymin + ymax)/2.0
    w = xmax - xmin
    h = ymax - ymin
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [ ]:
count=0
for file in glob('coco/data3/val/*.png'):
    #shutil.copy2(file, 'coco/train/images/' )
    #print(name)
    name = file.split('/')[-1]
    #print(name[:-4])
    img = cv2.imread(file, 0)
    im=Image.open(file)

    ret,thresh = cv2.threshold(img,30,255,1)
    #print(thresh.shape[0])
    contours,h = cv2.findContours(thresh,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    x, y, w, h = cv2.boundingRect(contours[1])
    #contours,h = cv2.findContours(thresh,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    direct = os.path.join('coco/data3/val/', f"{name[:-4]}.txt")
    with open(direct, "w") as text_file:
        for i in range(len(contours)):
            area = cv2.contourArea(contours[i])
            if area < 600000 and area > 100:
                x, y, w, h = cv2.boundingRect(contours[i])
                #print(name, area )
                ww, hh = x+w, y+h
                box = (x, y, ww, hh)
                #print(box)
                bbox = convert_labels(im.size, x,y,ww, hh)
                #print(im.size)
                print(name,bbox)
                text_file.write(f'0 {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}'+'\n')
    count+=1 
    text_file.close()

In [ ]:
img = cv2.imread('coco/data3/val/773854.png')
#print(img)
dh, dw, _ = img.shape

fl = open('coco/data3/val/773854.txt', 'r')
data = fl.readlines()
fl.close()
for dt in data:

    # Split string to float
    _, x, y, w, h = map(float, dt.split(' '))

    # Taken from https://github.com/pjreddie/darknet/blob/810d7f797bdb2f021dbe65d2524c2ff6b8ab5c8b/src/image.c#L283-L291
    # via https://stackoverflow.com/questions/44544471/how-to-get-the-coordinates-of-the-bounding-box-in-yolo-object-detection#comment102178409_44592380
    l = int((x - w / 2) * dw)
    r = int((x + w / 2) * dw)
    t = int((y - h / 2) * dh)
    b = int((y + h / 2) * dh)
    
    if l < 0:
        l = 0
    if r > dw - 1:
        r = dw - 1
    if t < 0:
        t = 0
    if b > dh - 1:
        b = dh - 1

    cv2.rectangle(img, (l, t), (r, b), (0, 0, 255), 3)

plt.imshow(img)
plt.show()